# Sort Spotify Playlist by Popularity

This notebook:

1. Fetches a playlist by ID
2. Sorts tracks by Spotify popularity (0–100, higher first by default)
3. Creates a new playlist with that order

It handles pagination, skips local files/episodes (no popularity), and adds tracks in 100-item batches.


## Setup

Install required packages if needed:

```bash
pip install spotipy python-dotenv
```

Create a Spotify app at https://developer.spotify.com/dashboard and set these env vars (or put them in a .env file):

- `SPOTIPY_CLIENT_ID`
- `SPOTIPY_CLIENT_SECRET`
- `SPOTIPY_REDIRECT_URI` (e.g., `http://localhost:8888/callback`)

The first run opens a browser to authorize. Scopes requested: playlist-read-private, playlist-modify-private, playlist-modify-public.


In [ ]:
import os
import time

import spotipy
from dotenv import load_dotenv
from spotipy.exceptions import SpotifyException
from spotipy.oauth2 import SpotifyOAuth

load_dotenv()  # Load .env file if present

In [ ]:
SCOPES = [
    "playlist-read-private",
    "playlist-modify-private",
    "playlist-modify-public",
]

## Helper Functions


In [ ]:
def backoff_sleep(retry: int) -> None:
    """Simple exponential backoff with jitter"""
    delay = min(2**retry, 30) + (0.25 * retry)
    time.sleep(delay)


def sp_client() -> spotipy.Spotify:
    """Create authenticated Spotify client"""
    auth_mgr = SpotifyOAuth(
        scope=" ".join(SCOPES),
        client_id=os.getenv("SPOTIPY_CLIENT_ID"),
        client_secret=os.getenv("SPOTIPY_CLIENT_SECRET"),
        redirect_uri=os.getenv("SPOTIPY_REDIRECT_URI"),
        open_browser=True,
        cache_path=os.path.join(os.path.expanduser("~"), ".cache-spotify-popularity-sort"),
        show_dialog=False,
    )
    return spotipy.Spotify(auth_manager=auth_mgr)

In [ ]:
def fetch_playlist(sp: spotipy.Spotify, playlist_id: str) -> dict[str, any]:
    """Fetch basic playlist metadata"""
    try:
        playlist = sp.playlist(
            playlist_id, fields="id,name,owner(id,display_name),public,tracks(total)"
        )
        return playlist
    except SpotifyException as e:
        if e.http_status == 404:
            raise ValueError(
                "Playlist not found or not accessible. If this is a personalized playlist (e.g., Daily Mix, Discover Weekly, On Repeat) or an invalid ID, Spotify returns 404. Try a public or owned playlist, or pass a full playlist URL."
            ) from e
        raise


def fetch_all_tracks(sp: spotipy.Spotify, playlist_id: str) -> list[dict[str, any]]:
    """Fetch all tracks from a playlist with pagination"""
    items: list[dict[str, any]] = []
    limit = 100
    offset = 0
    retries = 0
    total = None

    while True:
        try:
            page = sp.playlist_items(
                playlist_id,
                offset=offset,
                limit=limit,
                fields="items(added_at,track(id,name,artists(name),popularity,is_local,type,uri)),total,next",
            )
            retries = 0
        except SpotifyException as e:
            if e.http_status == 429 and retries < 5:
                retries += 1
                backoff_sleep(retries)
                continue
            raise

        if total is None:
            total = page.get("total", 0)

        items.extend(page.get("items", []))
        if len(items) >= total or not page.get("next"):
            break
        offset += limit

    return items

In [ ]:
def extract_track_entries(items: list[dict[str, any]]) -> tuple[list[dict[str, any]], list[str]]:
    """Extract valid tracks and skip non-tracks/local files"""
    valid = []
    skipped = []

    for it in items:
        tr = it.get("track")
        if not tr:
            skipped.append("Missing track object")
            continue

        # Skip episodes and local files (they don't have popularity / re-adding can fail)
        if tr.get("type") != "track":
            skipped.append(f"Non-track item: {tr.get('type')}")
            continue
        if tr.get("is_local"):
            skipped.append(f"Local file: {tr.get('name')}")
            continue
        if not tr.get("id"):
            skipped.append(f"No track ID: {tr.get('name')}")
            continue

        popularity = tr.get("popularity") or 0  # None -> 0
        artists = ", ".join(a["name"] for a in (tr.get("artists") or []))
        valid.append(
            {
                "id": tr["id"],
                "uri": tr["uri"],
                "name": tr.get("name", "Unknown"),
                "artists": artists,
                "popularity": popularity,
            }
        )

    return valid, skipped

In [ ]:
def sort_by_popularity(
    tracks: list[dict[str, any]], descending: bool = True
) -> list[dict[str, any]]:
    """Sort tracks by popularity"""
    return sorted(tracks, key=lambda t: t["popularity"], reverse=descending)


def chunked(seq: list[any], size: int) -> list[list[any]]:
    """Split a list into chunks of specified size"""
    return [seq[i : i + size] for i in range(0, len(seq), size)]

In [ ]:
def create_new_playlist(
    sp: spotipy.Spotify,
    user_id: str,
    name: str,
    public: bool,
    description: str | None = None,
) -> str:
    """Create a new playlist and return its ID"""
    retries = 0
    while True:
        try:
            pl = sp.user_playlist_create(
                user=user_id,
                name=name,
                public=public,
                description=description or "",
            )
            return pl["id"]
        except SpotifyException as e:
            if e.http_status in (429, 500, 502, 503, 504) and retries < 5:
                retries += 1
                backoff_sleep(retries)
                continue
            raise


def add_tracks(sp: spotipy.Spotify, playlist_id: str, uris: list[str]) -> None:
    """Add tracks to playlist in batches of 100"""
    for batch in chunked(uris, 100):
        retries = 0
        while True:
            try:
                sp.playlist_add_items(playlist_id, batch)
                break
            except SpotifyException as e:
                if e.http_status in (429, 500, 502, 503, 504) and retries < 5:
                    retries += 1
                    backoff_sleep(retries)
                    continue
                raise

## Main Sorting Function


In [ ]:
def sort_playlist(
    playlist_id: str,
    ascending: bool = False,
    new_name: str | None = None,
    force_public: bool | None = None,
):
    """
    Sort a Spotify playlist by popularity and create a new playlist.

    Args:
        playlist_id: Spotify playlist ID or full URL
        ascending: If True, sort from least to most popular (default: False)
        new_name: Name for the new playlist (default: '<orig> — sorted by popularity')
        force_public: If set, override the original playlist's visibility
    """
    # Create client
    sp = sp_client()

    # Get current user
    me = sp.current_user()
    user_id = me["id"]
    print(f"Authenticated as: {me.get('display_name', user_id)}")

    # Normalize playlist ID (handle URLs and URIs)
    if "open.spotify.com/playlist/" in playlist_id:
        playlist_id = playlist_id.split("playlist/")[1].split("?")[0]
    if playlist_id.startswith("spotify:playlist:"):
        playlist_id = playlist_id.split(":")[-1]

    # Fetch playlist metadata
    playlist = fetch_playlist(sp, playlist_id)
    orig_name = playlist["name"]
    orig_public = bool(playlist.get("public"))

    print(f"\nFetched playlist: {orig_name}")
    print(f"Visibility: {'public' if orig_public else 'private'}")
    print(f"Total tracks: {playlist['tracks']['total']}")

    # Fetch all tracks
    print("\nFetching all tracks...")
    items = fetch_all_tracks(sp, playlist_id)
    print(f"Found {len(items)} items")

    # Extract valid tracks
    tracks, skipped = extract_track_entries(items)
    if skipped:
        print(f"Skipping {len(skipped)} items (local files, episodes, or missing data)")

    if not tracks:
        print("No valid tracks with popularity found. Exiting.")
        return None

    print(f"Processing {len(tracks)} valid tracks")

    # Sort tracks
    sorted_tracks = sort_by_popularity(tracks, descending=not ascending)

    # Show top and bottom tracks
    print("\nTop 5 tracks by popularity:")
    for i, t in enumerate(sorted_tracks[:5], 1):
        print(f"  {i}. {t['name']} - {t['artists']} (popularity: {t['popularity']})")

    if len(sorted_tracks) > 5:
        print("\nBottom 5 tracks by popularity:")
        for i, t in enumerate(sorted_tracks[-5:], len(sorted_tracks) - 4):
            print(f"  {i}. {t['name']} - {t['artists']} (popularity: {t['popularity']})")

    # Prepare new playlist metadata
    final_name = new_name or f"{orig_name} — sorted by popularity"
    final_public = force_public if force_public is not None else orig_public
    desc = f"Auto-generated: {len(sorted_tracks)} tracks sorted by Spotify popularity ({'desc' if not ascending else 'asc'})."

    # Create and fill the new playlist
    print(f"\nCreating new playlist: {final_name}")
    new_playlist_id = create_new_playlist(sp, user_id, final_name, final_public, description=desc)

    print("Adding tracks to new playlist...")
    add_tracks(sp, new_playlist_id, [t["uri"] for t in sorted_tracks])

    print("\n✅ Successfully created playlist!")
    print(f"   Name: {final_name}")
    print(f"   ID: {new_playlist_id}")
    print(f"   URL: https://open.spotify.com/playlist/{new_playlist_id}")
    print(f"   Visibility: {'public' if final_public else 'private'}")
    print(f"   Tracks: {len(sorted_tracks)}")
    if skipped:
        print(f"   Skipped items: {len(skipped)}")

    return new_playlist_id

## Example Usage

Run the cell below with your playlist ID or URL:


In [ ]:
# Example: Sort a playlist by popularity (most popular first)
# Replace with your playlist ID or URL

# playlist URL: https://open.spotify.com/playlist/37i9dQZF1DZ06evO3Dj9e7?si=e3f49fffd4084adf
# playlist URL: https://open.spotify.com/playlist/1wrErf9WO9oNFxGQKWNm8S?si=69daeb8c26f74ba7
# playlist URL: https://open.spotify.com/playlist/1IV5FJDI9hDJtPZDBq5CKD?si=8e67f7bd8aff4eae

PLAYLIST_ID = "1IV5FJDI9hDJtPZDBq5CKD"

# Sort descending (most popular first), keep original visibility
new_playlist_id = sort_playlist(PLAYLIST_ID)

In [ ]:
# Example: Sort ascending (least popular first) and force private
# new_playlist_id = sort_playlist(
#     PLAYLIST_ID,
#     ascending=True,
#     force_public=False
# )

In [ ]:
# Example: Custom name
# new_playlist_id = sort_playlist(
#     PLAYLIST_ID,
#     new_name="My Playlist (popularity ordered)"
# )

## Interactive Mode

Use this cell for an interactive experience:


In [ ]:
def interactive_sort():
    """Interactive mode for sorting playlists"""
    playlist_input = input("Enter playlist ID or URL: ").strip()
    if not playlist_input:
        print("No playlist ID provided")
        return

    sort_order = input("Sort order (desc/asc) [desc]: ").strip().lower() or "desc"
    ascending = sort_order == "asc"

    custom_name = input("Custom name (press Enter to use default): ").strip()
    custom_name = custom_name if custom_name else None

    visibility = input("Visibility (public/private/keep) [keep]: ").strip().lower() or "keep"
    force_public = None
    if visibility == "public":
        force_public = True
    elif visibility == "private":
        force_public = False

    print("\n" + "=" * 50)
    sort_playlist(
        playlist_input, ascending=ascending, new_name=custom_name, force_public=force_public
    )


# Uncomment to run interactive mode
# interactive_sort()

## Notes

- **Popularity**: Available for tracks only, not podcast episodes or local files (those are skipped)
- **New Playlist**: This script creates a new playlist; it does not mutate the original
- **Rate Limits**: If you hit a rate limit (HTTP 429), the script automatically retries with exponential backoff
- **Authentication**: The first run will open a browser to authorize. Token is cached for future use
